### url_info

In [ ]:
### 상품 검색 및 필터링

from flask import Flask, url_for

app = Flask(__name__)

@app.route('/products')
def product_list():
    return '상품 목록'

with app.test_request_context():
    # 기본 상품 목록
    print(url_for('product_list'))  # /products

    # 검색어와 함께
    print(url_for('product_list', search='노트북'))  # /products?search=노트북

    # 검색어 + 가격 필터 + 정렬
    print(url_for('product_list',
                  search='노트북',
                  min_price=1000000,
                  max_price=2000000,
                  sort='price_desc'))
    # /products?search=노트북&min_price=1000000&max_price=2000000&sort=price_desc

/products
/products?search=%EB%85%B8%ED%8A%B8%EB%B6%81
/products?search=%EB%85%B8%ED%8A%B8%EB%B6%81&min_price=1000000&max_price=2000000&sort=price_desc


In [ ]:
from flask import Flask, request, render_template_string
from urllib.parse import unquote

app = Flask(__name__)

@app.route('/products')
def product_list():
    params = {k: unquote(v) for k, v in request.args.items()}

    table_html = """
    <h2>상품 검색 파라미터</h2>
    <table border="1">
        <tr><th>파라미터</th><th>값</th></tr>
        {% for key, value in params.items() %}
        <tr><td>{{ key }}</td><td>{{ value }}</td></tr>
        {% endfor %}
    </table>
    """
    return render_template_string(table_html, params=params)

# 테스트 URL:
# /products?search=노트북&min_price=1000000&max_price=2000000&sort=price_desc


In [ ]:
### 게시판 페이징 및 카테고리

@app.route('/board')
@app.route('/board/<int:page>')
def board():
    return '게시판'

with app.test_request_context():
    # 기본 게시판
    print(url_for('board'))  # /board

    # 페이지 + 카테고리
    print(url_for('board',
                  page=2,
                  category='notice'))  # /board?page=2&category=notice

    # 페이지 + 카테고리 + 검색
    print(url_for('board',
                  page=2,
                  category='notice',
                  search='공지',
                  sort='recent'))
    # /board?page=2&category=notice&search=공지&sort=recent

/board
/board/2?category=notice
/board/2?category=notice&search=%EA%B3%B5%EC%A7%80&sort=recent


In [ ]:
### 사용자 프로필 필터
@app.route('/users')
def user_list():
    return '사용자 목록'

with app.test_request_context():
    # 기본 사용자 목록
    print(url_for('user_list'))  # /users

    # 지역 필터
    print(url_for('user_list',
                  location='서울'))  # /users?location=서울

    # 상세 필터
    print(url_for('user_list',
                  location='서울',
                  age_min=20,
                  age_max=30,
                  gender='F',
                  status='active'))
    # /users?location=서울&age_min=20&age_max=30&gender=F&status=active

/users
/users?location=%EC%84%9C%EC%9A%B8
/users?location=%EC%84%9C%EC%9A%B8&age_min=20&age_max=30&gender=F&status=active


In [ ]:
# 이벤트 캘린더
@app.route('/events')
def event_list():
    return '이벤트 목록'

with app.test_request_context():
    # 기본 이벤트 목록
    print(url_for('event_list'))  # /events

    # 날짜 필터
    print(url_for('event_list',
                  date='2024-01'))  # /events?date=2024-01

    # 상세 필터
    print(url_for('event_list',
                  start_date='2024-01-01',
                  end_date='2024-12-31',
                  type='concert',
                  location='seoul'))
    # /events?start_date=2024-01-01&end_date=2024-12-31&type=concert&location=seoul

/events
/events?date=2024-01
/events?start_date=2024-01-01&end_date=2024-12-31&type=concert&location=seoul


In [ ]:
### 레스토랑 검색
@app.route('/restaurants')
def restaurant_list():
    return '레스토랑 목록'

with app.test_request_context():
    # 기본 레스토랑 목록
    print(url_for('restaurant_list'))  # /restaurants

    # 간단한 필터
    print(url_for('restaurant_list',
                  cuisine='korean'))  # /restaurants?cuisine=korean

    # 상세 검색
    print(url_for('restaurant_list',
                  cuisine='korean',
                  district='gangnam',
                  price_range='20000-50000',
                  rating_min=4,
                  open_now=True))
    # /restaurants?cuisine=korean&district=gangnam&price_range=20000-50000&rating_min=4&open_now=True

/restaurants
/restaurants?cuisine=korean
/restaurants?cuisine=korean&district=gangnam&price_range=20000-50000&rating_min=4&open_now=True


In [ ]:
from flask import Flask, request, url_for

app = Flask(__name__)

# 1. GET 요청 테스트
with app.test_request_context('/search?q=flask'):
    assert request.args.get('q') == 'flask'
    assert request.method == 'GET'

# 2. POST 요청 테스트
with app.test_request_context('/login', method='POST',
                            data={'username': 'test'}):
    assert request.form.get('username') == 'test'
    assert request.method == 'POST'

# 3. JSON 데이터 테스트
with app.test_request_context('/api/users',
                            method='POST',
                            json={'name': 'John'}):
    assert request.get_json()['name'] == 'John'

# 4. 헤더 테스트
with app.test_request_context('/api',
                            headers={'Authorization': 'Bearer token'}):
    assert request.headers['Authorization'] == 'Bearer token'

# 5. URL 생성 테스트
@app.route('/user/<username>')
def user_profile(username):
    return f"Profile of {username}"

with app.test_request_context():
    url = url_for('user_profile', username='test')
    assert url == '/user/test'

In [ ]:
### 동적 URL 구성

from flask import Flask

app = Flask(__name__)

@app.route('/')
@app.route('/home')
def home():
    return 'Hello, World!'

# 동적URL: string, int, float (함수의 인자로 사용)
@app.route('/user/<user_name>/<int:user_id>')
def user(user_name, user_id):
    return f'Hello, {user_name}({user_id})!'

if __name__ == '__main__':
    app.run(debug=True)